# PolyA Estmated Lenghts from Dorado

In [1]:
def read_sam_file(sam_file):
    """
    Reads a SAM file, extracts read IDs and 'pt:i:' values for sequences aligned to 'eGFP\'.
    
    Parameters:
    - sam_file (str): Path to the SAM file.
    
    Returns:
    - pt_values (list): A list of extracted 'pt:i:' values.
    """
    read_ids = set()
    pt_values = []  # List to store pt:i: values

    with open(sam_file, "r") as file:
        for line in file:
            if line.startswith("@"):  # Skip header lines
                continue

            fields = line.strip().split("\t")  # Split the SAM fields
            read_id = fields[0]  # First column is the read ID
            ref_name = fields[2]  # Third column (reference name)

            # Filter only reads that align to "eGFP\"
            if ref_name != "eGFP\\":
                continue
            
            # Extract the 'pt:i:' value
            for field in fields:
                if field.startswith("pt:i:"):
                    pt_value = int(field.split(":")[2])  # Convert to integer
                    pt_values.append(pt_value)
                    break  # Stop searching once found
            
            read_ids.add(read_id)

    # Print total number of matching reads
    print(f"Total number of filtered read IDs: {len(read_ids)}")

    return pt_values

In [2]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_est_length(pt_values, sam_file, pdf):
    """
    Plots the distribution of 'pt:i:' values using a histogram and saves multiple plots to a single PDF page.
    
    Parameters:
    - pt_values_list (list of lists): List of extracted 'pt:i:' values for each SAM file.
    - sam_files (list): List of SAM file names (for titles).
    - pdf (PdfPages): PDF file handler for saving plots.
    """
    
    if not pt_values:
        print(f"No pt:i: values to plot for {sam_file}.")
        return

    plt.figure(figsize=(10, 5))
    plt.hist(pt_values, bins=50, color='skyblue', edgecolor='black')
    plt.xlabel("pt:i: Value")
    plt.ylabel("Frequency")
    plt.title(f"Distribution of Estimated Lengths\n({sam_file})")
    plt.xlim(-10, 260)
    plt.ylim(0, 161000)

    # Save the plot to the PDF
    pdf.savefig()
    plt.close()

In [3]:
# List of your SAM files
sam_files = [
    "egfp_a60_30.sam", 
    "egfp_a60_60.sam", 
    "egfp_a60_unmod.sam",
    "egfp_a120_1mod.sam", 
    "egfp_a120_2mod.sam", 
    "egfp_a120_4mod.sam", 
    "egfp_a120_unmod.sam"
]

# Create a single PDF to store all plots
output_pdf = "Distribution_of_polyA_estimated_lengths.pdf"

# Open the PDF to save the plots
with PdfPages(output_pdf) as pdf:
    for sam_file in sam_files:
        pt_values = read_sam_file(sam_file)
        plot_est_length(pt_values, sam_file, pdf)

print(f"All histograms saved in {output_pdf}")

Total number of filtered read IDs: 337608
Total number of filtered read IDs: 242867
Total number of filtered read IDs: 303563
Total number of filtered read IDs: 152670
Total number of filtered read IDs: 138440
Total number of filtered read IDs: 229452
Total number of filtered read IDs: 207288
All histograms saved in Distribution_of_polyA_estimated_lengths.pdf


In [4]:
def plot_est_length_multiple(pt_values_list, sam_files, pdf):
    """
    Plots the distribution of 'pt:i:' values using a histogram and saves multiple plots to a single PDF page.
    
    Parameters:
    - pt_values_list (list of lists): List of extracted 'pt:i:' values for each SAM file.
    - sam_files (list): List of SAM file names (for titles).
    - pdf (PdfPages): PDF file handler for saving plots.
    """
    # Fixed layout for 7 plots: 3 rows, 3 columns
    rows, cols = 3, 3

    # Create a new figure to hold multiple subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 10))

    # Flatten the axes array for easier indexing
    axes = axes.flatten()

    # Loop through each SAM file and plot its pt:i: distribution
    for i, (pt_values, sam_file) in enumerate(zip(pt_values_list, sam_files)):
        ax = axes[i]
        if not pt_values:
            ax.text(0.5, 0.5, f"No pt:i: values to plot for {sam_file}", ha="center", va="center")
        else:
            ax.hist(pt_values, bins=50, color='skyblue', edgecolor='black')
            ax.set_xlabel("pt:i: Value")
            ax.set_ylabel("Frequency")
            ax.set_title(f"Distribution of Estimated Lengths\n({sam_file})")
            ax.set_xlim(-10, 160)
            ax.set_ylim(0, 170000)

    # Hide unused subplots (the last 2 in a 3x3 grid)
    for j in range(len(pt_values_list), len(axes)):
        axes[j].axis('off')

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)

In [5]:
# List of your SAM files
sam_files = [
    "egfp_a60_30.sam", 
    "egfp_a60_60.sam", 
    "egfp_a60_unmod.sam",
    "egfp_a120_1mod.sam", 
    "egfp_a120_2mod.sam", 
    "egfp_a120_4mod.sam", 
    "egfp_a120_unmod.sam"
]

# Create a single PDF to store all plots
output_pdf = "Distribution_of_polyA_estimated_lengths.pdf"

# Open the PDF to save the plots
with PdfPages(output_pdf) as pdf:
    pt_values_list = []
    for sam_file in sam_files:
        pt_values = read_sam_file(sam_file)
        pt_values_list.append(pt_values)
    
    # Plot and save all the plots to a single page in the PDF
    plot_est_length_multiple(pt_values_list, sam_files, pdf)

print(f"All histograms saved in {output_pdf}")

Total number of filtered read IDs: 337608
Total number of filtered read IDs: 242867
Total number of filtered read IDs: 303563
Total number of filtered read IDs: 152670
Total number of filtered read IDs: 138440
Total number of filtered read IDs: 229452
Total number of filtered read IDs: 207288
All histograms saved in Distribution_of_polyA_estimated_lengths.pdf
